In [1]:
# pip install -r requirements.txt

In [2]:
from pandas import read_csv

In [3]:
x = read_csv("MOS Data.csv")
z = x.sample(n=10)
z

,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet)
15,16,57,13,662,125
29,30,85,20,570,195
2,3,31,6,550,60
3,4,33,7,400,65
7,8,41,9,680,85
20,21,67,15,633,150
5,6,37,8,660,75
19,20,65,15,640,145
28,29,83,19,577,190
0,1,27,5,500,50


In [4]:
n = list(z.iloc[:,1])
f = n[0]
x = []
for i in n:
    x.append( i/f)
z["Modulus Ratio (N)"] = x
z

,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet),Modulus Ratio (N)
15,16,57,13,662,125,1.000000
29,30,85,20,570,195,1.491228
2,3,31,6,550,60,0.543860
3,4,33,7,400,65,0.578947
7,8,41,9,680,85,0.719298
20,21,67,15,633,150,1.175439
5,6,37,8,660,75,0.649123
19,20,65,15,640,145,1.140351
28,29,83,19,577,190,1.456140
0,1,27,5,500,50,0.473684


In [35]:
h = [0.04]*10
z["Height"] = h
b = [0.2] * 10
z["Breadth"] = b

eq = [i*0.2 for i in x]
z["Equivalent Breadth"] = eq

eqar = [i*0.04 for i in eq]
z["Equivalent Area"] = eqar

y = [ 0.38 - 0.04*i for i in range(0,10,1)]
z["y"] = y

yar = [ eqar[i] * y[i] for i in range(len(x))]
sum_yar = sum(yar)
sum_ar = sum(eqar)
yc = sum_yar/sum_ar
print(yc)
z

0.2028897338403042


,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet),Modulus Ratio (N),Height,Breadth,Equivalent Breadth,Equivalent Area,y
15,16,57,13,662,125,1.000000,0.04,0.2,0.200000,0.008000,0.38
29,30,85,20,570,195,1.491228,0.04,0.2,0.298246,0.011930,0.34
2,3,31,6,550,60,0.543860,0.04,0.2,0.108772,0.004351,0.30
3,4,33,7,400,65,0.578947,0.04,0.2,0.115789,0.004632,0.26
7,8,41,9,680,85,0.719298,0.04,0.2,0.143860,0.005754,0.22
20,21,67,15,633,150,1.175439,0.04,0.2,0.235088,0.009404,0.18
5,6,37,8,660,75,0.649123,0.04,0.2,0.129825,0.005193,0.14
19,20,65,15,640,145,1.140351,0.04,0.2,0.228070,0.009123,0.10
28,29,83,19,577,190,1.456140,0.04,0.2,0.291228,0.011649,0.06
0,1,27,5,500,50,0.473684,0.04,0.2,0.094737,0.003789,0.02


In [38]:
yi = [i-yc for i in y]
z["yi"] = yi
z

,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet),Modulus Ratio (N),Height,Breadth,Equivalent Breadth,Equivalent Area,y,yi
15,16,57,13,662,125,1.000000,0.04,0.2,0.200000,0.008000,0.38,0.17711
29,30,85,20,570,195,1.491228,0.04,0.2,0.298246,0.011930,0.34,0.13711
2,3,31,6,550,60,0.543860,0.04,0.2,0.108772,0.004351,0.30,0.09711
3,4,33,7,400,65,0.578947,0.04,0.2,0.115789,0.004632,0.26,0.05711
7,8,41,9,680,85,0.719298,0.04,0.2,0.143860,0.005754,0.22,0.01711
20,21,67,15,633,150,1.175439,0.04,0.2,0.235088,0.009404,0.18,-0.02289
5,6,37,8,660,75,0.649123,0.04,0.2,0.129825,0.005193,0.14,-0.06289
19,20,65,15,640,145,1.140351,0.04,0.2,0.228070,0.009123,0.10,-0.10289
28,29,83,19,577,190,1.456140,0.04,0.2,0.291228,0.011649,0.06,-0.14289
0,1,27,5,500,50,0.473684,0.04,0.2,0.094737,0.003789,0.02,-0.18289


In [40]:
I = [eq[i]*pow(0.04,3)/12 + eq[i]*0.04*pow(yi[i],2) for i in range(10)]
z["I"] = I
z

,Material \nID,Elastic modulus \n(GPa),Yield strength \n(MPa),Density \n(kg/m3),Price (INR/m2 of sheet),Modulus Ratio (N),Height,Breadth,Equivalent Breadth,Equivalent Area,y,yi,I
15,16,57,13,662,125,1.000000,0.04,0.2,0.200000,0.008000,0.38,0.17711,0.000252
29,30,85,20,570,195,1.491228,0.04,0.2,0.298246,0.011930,0.34,0.13711,0.000226
2,3,31,6,550,60,0.543860,0.04,0.2,0.108772,0.004351,0.30,0.09711,0.000042
3,4,33,7,400,65,0.578947,0.04,0.2,0.115789,0.004632,0.26,0.05711,0.000016
7,8,41,9,680,85,0.719298,0.04,0.2,0.143860,0.005754,0.22,0.01711,0.000002
20,21,67,15,633,150,1.175439,0.04,0.2,0.235088,0.009404,0.18,-0.02289,0.000006
5,6,37,8,660,75,0.649123,0.04,0.2,0.129825,0.005193,0.14,-0.06289,0.000021
19,20,65,15,640,145,1.140351,0.04,0.2,0.228070,0.009123,0.10,-0.10289,0.000098
28,29,83,19,577,190,1.456140,0.04,0.2,0.291228,0.011649,0.06,-0.14289,0.000239
0,1,27,5,500,50,0.473684,0.04,0.2,0.094737,0.003789,0.02,-0.18289,0.000127
